In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from datetime import datetime

In [3]:
from ecephys.xrsig import get_emg
from sglxarray import load_trigger
import ecephys_analyses as ea

In [4]:
EMG_OPTS = dict(target_sf=20, window_size=25.0, wp=[300, 600], ws=[275, 625], gpass=1, gstop=60, ftype='butter')

In [5]:
def get_condition_emg(subject, experiment, alias, probe):
    emg_chans = ea.get_channels(subject, experiment, probe, "emg_from_lfp")
    
    bin_paths = ea.get_lfp_bin_paths(subject, experiment, alias, probe=probe)
    emg_paths = ea.get_analysis_counterparts(bin_paths, "emg.nc", subject)
    #bin_paths = ea.get_sglx_style_datapaths(subject=subject, experiment=experiment, condition=condition, ext="lf.bin")
    #emg_paths = ea.get_sglx_style_datapaths(subject=subject, experiment=experiment, condition=condition, ext="emg.nc")

    for bin_path, emg_path in zip(bin_paths, emg_paths):
        sig = load_trigger(bin_path, emg_chans)
        emg = get_emg(sig, **EMG_OPTS)
        
        # Create parent directories if they do not already exist.
        Path(emg_path).parent.mkdir(parents=True, exist_ok=True)  
        emg.to_netcdf(emg_path)
        emg.close()
        
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"{current_time}: Finished {str(bin_path)}")

In [8]:
get_condition_emg("Adrian", "sleep_homeostasis", "patch3", "imec1")

nChan: 385, nFileSamp: 12750840
Filtering LFP with wp=[300, 600], ws=[275, 625], gpass=1, gstop=60,filter type=butter
Computing EMG from filtered LFP...
target sf = 20, window size = 25.0, LFP sf=2500.0113678480425, LFP nchans = 3




Done!
11:34:29: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX11-Adrian/8-31-2021/SpikeGLX/8-31-2021_g1/8-31-2021_g1_imec1/8-31-2021_g1_t1.imec1.lf.bin
